In [53]:
import pandas as pd

import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [54]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    print(f'Number of columns: {df.shape[1]}')
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    print(f'Standard Deviation: {df.duration.std()}')

    df_outliers = df.copy()

    df_outliers = df_outliers[(df_outliers.duration >= 1) & (df_outliers.duration <= 60)]
    print(f'Fraction of the records left after outliers dropped: {df_outliers.shape[0] / df.shape[0]}')

    categorical = ['PULocationID', 'DOLocationID']
    df_outliers[categorical] = df_outliers[categorical].astype(str)
    
    return df_outliers

In [26]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Number of columns: 19
Standard Deviation: 42.59435124195458
Fraction of the records left after outliers dropped: 0.9812202822125979
Number of columns: 19
Standard Deviation: 42.84210176105113
Fraction of the records left after outliers dropped: 0.9800944077722545


In [46]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [47]:
print(f'Dimensionality of this matrix: {X_train.shape[1]}')

Dimensionality of this matrix: 515


In [48]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [51]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261822035489

In [52]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811821332387183